© 2025 Vanargo · Лицензия: MIT. См. файл `LICENSE` в корне репозитория.

# --- Fairness & Explainability: setup --- #

**Цель.** Оценить справедливость и качество финальной модели на тесте, выбрать рабочий порог `t*` с учетом fairness-ограничений, проверить калибровку по группам и объяснить вклад признаков.

**Входы (из `01_data_loading_and_eda.ipynb`/`02_modeling.ipynb`):**
1. `X_test_enc`, `y_true_test`, `feature_names` - подготовка из `02_modeling.ipynb`.
2. `y_proba_best` - прогнозы лучшей модели на тесте.
3. `X_test_sens` - чувствительные признаки (`sex`, `race`, `age_group`).

**Выходы (в `data/reports/figures_03/`, `data/artifacts`):**
1. Пороговый скан и Pareto: `accuracy_f1_vs_threshold.png`, `dp_vs.threshold.png`, `Pareto_f1_vs_dp.png`.
2. Метрики по группам при `t=0.5` и `t*`: таблицы CSV и бар-чарты.
3. Калибровка по группам: `calibration_*.png` и сводная ЕСЕ.
4. Explainability: топ-фичи (SHAP/permutation), dependence-plots.
5. Служебные артефакты: выбранный `t*`, параметры пост-обработки (если применяли `ThresholdOptimizer`).

**Содержимое ноутбука:**
1. Sanity-checks: размеры, NaN, распределение `y_proba`.
2. Базовые метрики при `t=0.5`.
3. Fairness по группам при `t=0.5`.
4. Пороговый скан -> выбор `t*` (компромисс качество <-> справедливость).
5. Пост-обработка (`ThresholdOptimizer`: `demographic_parity` / `equalized_odds`).
6. Калибровка вероятностей по группам (reliability curves, ECE).
7. Explainability: глобально и локально.
8. Риски, ограничения, рекомендации. Экспорт артефактов.

**Требования к окружению и путям:**
1. `ROOT`, `ART_DIR`, `REPORTS_DIR`, `FIG_DIR_03 = REPORTS_DIR/'    FIGURES_03'` должны существовать.
2. Все пути и объекты загружаются через единый загрузчик артефактов из `02_modeling.ipynb`. 

In [ ]:
# --- Project paths bootstrаp --- #

from __future__ import annotations

from pathlib import Path

# попытка использовать штатный модуль путей проекта #
try:
    from paths import (
        ART_DIR,
        DATA_DIR,
        INT_DIR,
        MODELS_DIR,
        NB_DIR,
        PROC_DIR,
        RAW_DIR,
        REPORTS_DIR,
        ROOT,
    )

    print(f"[paths] ROOT = {ROOT}")
except Exception as e:
    # fallback: минимально достаточные пути без побочных эффектов
    ROOT = Path.cwd()
    DATA_DIR = ROOT / "data"
    RAW_DIR = ROOT / "raw"
    INT_DIR = ROOT / "interim"
    PROC_DIR = ROOT / "processed"
    ART_DIR = ROOT / "artifacts"
    REPORTS_DIR = ROOT / "reports"
    MODELS_DIR = ROOT / "models"
    NB_DIR = ROOT / "notebooks"
    print(f"[paths:fallback] ROOT = {ROOT} | reason: {e!r}")

In [ ]:
# --- Fairness plotting utils & output dirs --- #

import matplotlib.pyplot as plt

# использование paths.py; при сбое - fallback #
try:
    _ = REPORTS_DIR
except NameError:
    try:
        from paths import REPORTS_DIR as _REPORTS_DIR
    except Exception:
        _REPORTS_DIR = ROOT / "data" / "reports"
    REPORTS_DIR = globals().get("REPORTS_DIR", _REPORTS_DIR)

FIG_DIR_03 = REPORTS_DIR / "figures_03"
FIG_DIR_03.mkdir(parents=True, exist_ok=True)


def _ensure_png(name: str) -> str:
    return name if name.lower().endswith(".png") else f"{name}.png"


def save_fig(name: str, fig=None, dpi: int = 200, close: bool = True):
    """
    Сохранение фигуры в `reports/figures_03/`
    name: имя файла, можно без .png
    fig: matplotlib.figure.Figure или None -> взять текущую.
    """
    if not isinstance(name, str) or not name:
        raise TypeError('save_fig: "name" должен быть непустой строкой.')

    # импорт pyplot только при необходимости #
    if fig is None:
        import matplotlib.pyplot as plt

        fig = plt.gcf()
    else:
        # простая проверка интерфейса
        if not hasattr(fig, "savefig"):
            raise TypeError('save_fig: "fig" должен иметь метод savefig.')

    fname = _ensure_png(name)
    path = FIG_DIR_03 / fname

    # сохранение #
    fig.savefig(path, dpi=dpi, bbox_inches="tight")

    # аккуратное закрытие только если явно просили #
    if close:
        try:
            import matplotlib.pyplot as plt

            plt.close(fig)
        except Exception:
            pass

    print(f"[saved] {path}")
    return Path

In [ ]:
# --- Fairness & Explainability: setup --- #

import warnings

# централизованное подавление предупреждений #
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

FAIRLEARN_OK = False
THRESH_OPT_OK = False
SHAP_OK = False

# fairlearn и компоненты #
try:
    from fairlearn.metrics import (
        demographic_parity_difference,
        equalized_odds_difference,
    )
    from fairlearn.postprocessing import ThresholdOptimizer

    FAIRLEARN_OK = True
    THRESH_OPT_OK = True
    print("[ok] fairlearn импортирован.")
except Exception as e:
    print(f"[warn] fairlearn недоступен: {e!r}")

# SHAP (глобальная интерпретируемость) #
try:
    import shap

    SHAP_OK = True
    print("[ok] shap импортирован.")
except Exception as e:
    print(f"[warn] shap недоступен: {e!r}")

# служебный флаг-индикатор готовности Explainability/Fairness #
EXPL_OK = FAIRLEARN_OK or SHAP_OK
print(f"[setup] FAIRLEARN_OK={FAIRLEARN_OK}, SHAP_OK={SHAP_OK}")

In [ ]:
# --- Notebook preamble: silence & style --- #

# autoreload только если доступен IPython #
try:
    ip = get_ipython()
    if ip:
        ip.run_line_magic("load_ext", "autoreload")
        ip.run_line_magic("autoreload", "2")
except Exception:
    pass

import logging

# приглушение шумных логгеров #
for name in ("matplotlib", "numba"):
    try:
        logging.getLogger(name).setLevel(logging.ERROR)
    except Exception:
        pass

# единый стиль графиков #
plt.rcParams.update(
    {
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.grid": True,
        "grid.alpha": 0.25,
        "figure.figsize": (6.5, 4.0),
        "axes.titlesize": 13,
        "axes.labelsize": 11,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10,
    }
)

print("[preamble] style set.")

In [ ]:
# --- Paths & unified artifact loader --- #

import json
from pathlib import Path

import joblib
import numpy as np
import pandas as pd

# базовые пути из проекта; при ошибке — fallback к структуре data/* #
try:
    from paths import ART_DIR as _ART_DIR
    from paths import MODELS_DIR as _MODELS_DIR
    from paths import REPORTS_DIR as _REPORTS_DIR
    from paths import ROOT
except Exception:
    ROOT = Path.cwd()
    while (
        not any((ROOT / m).exists() for m in (".git", "pyproject.toml", "README.md"))
        and ROOT.parent != ROOT
    ):
        ROOT = ROOT.parent
    _ART_DIR = ROOT / "data" / "artifacts"
    _MODELS_DIR = ROOT / "data" / "models"
    _REPORTS_DIR = ROOT / "data" / "reports"

# NB_DIR от реального ROOT, без несуществующего _ROOT #
NB_DIR = (ROOT / "notebooks") if "ROOT" in locals() else (Path.cwd() / "notebooks")

ART_DIR = _ART_DIR
MODELS_DIR = _MODELS_DIR
REPORTS_DIR = _REPORTS_DIR

CANDIDATE_ART_DIRS = [ART_DIR, NB_DIR / "data" / "artifacts"]
CANDIDATE_MODEL_DIRS = [MODELS_DIR, ART_DIR, NB_DIR / "data" / "models"]
FIG_DIR_02 = REPORTS_DIR / "figures_02"
FIG_DIR_03 = REPORTS_DIR / "figures_03"


def _pick_file(names: list[str], dirs: list[Path]) -> Path | None:
    for d in dirs:
        for n in names:
            p = Path(d) / n
            if p.exists():
                return p
    return None


# loaders: core #


def load_feature_names():
    p = _pick_file(
        ["feature_names.npy", "feature_names.json", "feature_names.txt", "feature_names.csv"],
        CANDIDATE_ART_DIRS,
    )
    if p is None:
        return None, None
    if p.suffix == ".npy":
        return np.load(p, allow_pickle=True).tolist(), p
    if p.suffix == ".json":
        return json.loads(p.read_text(encoding="utf-8")), p
    if p.suffix == ".txt":
        return [
            line.strip() for line in p.read_text(encoding="utf-8").splitlines() if line.strip()
        ], p
    if p.suffix == ".csv":
        return pd.read_csv(p, header=None)[0].tolist(), p
    return None, None


def load_X_test_enc():
    for nm in ["X_test_enc.npy", "X_test_enc.csv", "X_test.npy"]:
        p = _pick_file([nm], CANDIDATE_ART_DIRS)
        if p:
            if p.suffix == ".npy":
                return np.load(p, allow_pickle=False), p
            if p.suffix == ".csv":
                return pd.read_csv(p).to_numpy(), p
    return None, None


def load_X_test_sens():
    p = _pick_file(
        ["X_test_sensitive.csv", "X_test_sens.csv", "X_test_sensitive.parquet"], CANDIDATE_ART_DIRS
    )
    if p is None:
        return None, None
    if p.suffix == ".parquet":
        return pd.read_parquet(p), p
    if p.suffix == ".csv":
        return pd.read_csv(p), p
    return None, None


def load_y_true_pred_proba():
    y_true_p = _pick_file(["y_true_test.npy", "y_true.npy", "y_test.npy"], CANDIDATE_ART_DIRS)
    y_pred_p = _pick_file(["y_pred_best.npy", "y_pred.npy"], CANDIDATE_ART_DIRS)
    y_proba_p = _pick_file(["y_proba_best.npy", "y_proba.npy"], CANDIDATE_ART_DIRS)

    def _load(p):
        if p is None:
            return None
        if p.suffix == ".npy":
            return np.load(p, allow_pickle=False)
        if p.suffix == ".csv":
            return pd.read_csv(p, header=None).iloc[:, 0].to_numpy()
        return None

    return (
        _load(y_true_p),
        y_true_p,
        _load(y_pred_p),
        y_pred_p,
        _load(y_proba_p),
        y_proba_p,
    )


def load_best_model():
    cand = _pick_file(
        [
            "model_best.joblib",
            "LGBM_best.joblib",
            "lgb_best.joblib",
            "XGBoost_ES_best.joblib",
            "XGBoost_ES.joblib",
        ],
        CANDIDATE_MODEL_DIRS,
    )
    if cand is None:
        return None, None
    try:
        return joblib.load(cand), cand
    except Exception as e:
        print(f"[warn] cannot load {cand}: {type(e).__name__}: {e}")
        return None, cand


# loaders: extras we will use in 03 #


def load_results_df():
    p = _pick_file(["results_df.csv"], CANDIDATE_ART_DIRS)
    return (pd.read_csv(p), p) if p and p.exists() else (None, None)


def load_results_summary():
    p = _pick_file(["results_summary.csv"], CANDIDATE_MODEL_DIRS)
    return (pd.read_csv(p), p) if p and p.exists() else (None, None)


def load_export_meta():
    p = _pick_file(["export_meta.json"], CANDIDATE_ART_DIRS)
    return (json.loads(p.read_text(encoding="utf-8")), p) if p and p.exists() else (None, None)


def load_test_groups():
    p = _pick_file(["test_groups.csv"], CANDIDATE_ART_DIRS)
    return (pd.read_csv(p), p) if p and p.exists() else (None, None)


def load_y_test_optional():
    p = _pick_file(["y_test.npy"], CANDIDATE_ART_DIRS)
    return (np.load(p, allow_pickle=False), p) if p and p.exists() else (None, None)


def load_y_pred_050_optional():
    p = _pick_file(["y_pred_050.npy"], CANDIDATE_ART_DIRS)
    return (np.load(p, allow_pickle=False), p) if p and p.exists() else (None, None)


def load_y_score_optional():
    p = _pick_file(["y_score.npy"], CANDIDATE_ART_DIRS)
    return (np.load(p, allow_pickle=False), p) if p and p.exists() else (None, None)


def list_figures02():
    if not FIG_DIR_02.exists():
        return []
    return sorted([p for p in FIG_DIR_02.glob("*.png")])


# unified summary #


def load_artifacts_summary():
    fn, p_fn = load_feature_names()
    Xte, p_xe = load_X_test_enc()
    Xsens, p_xs = load_X_test_sens()
    y_true, p_yt, y_pred, p_yp, y_proba, p_ypr = (*load_y_true_pred_proba(),)
    y_test_opt, p_ytest = load_y_test_optional()
    y_pred_050_opt, p_y050 = load_y_pred_050_optional()
    y_score_opt, p_ys = load_y_score_optional()
    results_df, p_resdf = load_results_df()
    results_sum, p_ressum = load_results_summary()
    meta, p_meta = load_export_meta()
    groups, p_groups = load_test_groups()
    mdl, p_m = load_best_model()
    figs02 = list_figures02()
    return {
        "feature_names": (fn, p_fn),
        "X_test_enc": (Xte, p_xe),
        "sensitive": (Xsens, p_xs),
        "y_true": (y_true, p_yt),
        "y_pred": (y_pred, p_yp),
        "y_proba": (y_proba, p_ypr),
        "y_test_opt": (y_test_opt, p_ytest),
        "y_pred_050_opt": (y_pred_050_opt, p_y050),
        "y_score_opt": (y_score_opt, p_ys),
        "results_df": (results_df, p_resdf),
        "results_summary": (results_sum, p_ressum),
        "export_meta": (meta, p_meta),
        "test_groups": (groups, p_groups),
        "model": (mdl, p_m),
        "figures02": (figs02, FIG_DIR_02 if figs02 else None),
    }

In [ ]:
# --- Единый вызов после ячейки инициализации --- #

A = load_artifacts_summary()

model, model_path = A["model"]
feature_names, feature_names_path = A["feature_names"]
X_test_enc, X_test_enc_path = A["X_test_enc"]
X_test_sensitive, sens_path = A["sensitive"]
results_df, results_path = A["results_df"]
test_groups, groups_path = A["test_groups"]
y_true, y_true_path = A["y_true"]
y_proba, y_proba_path = A["y_proba"]
y_pred, y_pred_path = A["y_pred"]

print("[ok] model:", model_path.name)
for name, p in [
    ("feature_names", feature_names_path),
    ("X_test_enc", X_test_enc_path),
    ("sensitive", sens_path),
    ("results_df", results_path),
    ("test_groups", groups_path),
    ("y_true", y_true_path),
    ("y_proba", y_proba_path),
    ("y_pred", y_pred_path),
]:
    print(f"[{'ok' if p else 'miss'}] {name}:", (p.name if p else None))

# алиасы под старые имена для совместимости #
X_test_sens = X_test_sensitive
y_true_test = y_true
y_proba_best = y_proba
y_pred_best = y_pred

In [ ]:
# --- sanity-report из unified loader --- #

assert "A" in globals()

for name in [
    "model",
    "feature_names",
    "X_test_enc",
    "sensitive",
    "results_df",
    "results_summary",
    "test_groups",
    "y_true",
    "y_proba",
    "y_test_opt",
    "y_pred_050_opt",
    "y_score_opt",
    "export_meta",
    "figures02",
]:
    val, p = A[name]
    shape = getattr(val, "shape", None)
    if isinstance(val, list | tuple) and not shape:
        shape = f"len={len(val)}"
    print(f"[{name:>14}] path={getattr(p, 'name', None)} shape={shape}")

In [ ]:
# --- Provenance из export_meta.json --- #

meta, p_meta = A["export_meta"]
if meta:
    print(f"[meta] best_model={meta.get('best_model')} | ts={meta.get('timestamp')}")
    arts = meta.get("artifacts") or []
    print(f"[meta] listed artifacts: {len(arts)}")
else:
    print("[meta] export_meta.json not found.")

In [ ]:
# --- Leaderboard (results_summary.csv) --- #

res_sum, p_sum = A["results_summary"]

if res_sum is not None:
    df = res_sum.copy()
    # упорядочивание по test_f1, затем roc_auc #
    cols = [
        c
        for c in [
            "model",
            "test_f1",
            "test_roc_auc",
            "test_precision",
            "test_recall",
            "test_accuracy",
        ]
        if c in df.columns
    ]
    df = df.sort_values(
        by=[c for c in ["test_f1", "test_roc_auc"] if c in df.columns], ascending=False
    )
    display(df[cols].head(10))
else:
    print("[skip] results_summary.csv absent.")

In [ ]:
# --- Consistency check: y_true vs y_test --- #

y_true, _ = A["y_true"]
y_test_opt, p_ytest = A["y_test_opt"]
if y_true is not None and y_test_opt is not None:
    same = np.array_equal(y_true, y_test_opt)
    print(f"[check] y_true vs y_test: {'OK' if same else 'MISMATCH'}")
    if not same:
        raise RuntimeError("y_true_test.npy и y_test.npy различаются.")
else:
    print("[check] y_test optional not found -> skip.")

# --- sanity-check: размеры, NaN, распределение y_proba --- #

**Цель.** Проверить согласованность входных данных и корректность вероятностных прогнозов, прежде чем проводить анализ справедливости и объяснимости.

**Этапы проверки:**
1. Размерности. Убедиться, что длины `X_test_enc`, `X_test_sens`, `y_true_test`, `y_proba_best` совпадают. Несовпадение сигнализирует о нарушении консистентности между признаками, метками и предсказаниями модели.
2. Пропуски. Проверить наличие `NaN` в `y_proba_best` и чувствительных признаках. Их присутствие может исказить распределения и метрики fairness.
3. Распределение вероятностей:
    - вывести базовую статистику (`min`, `max`, `mean`, `std`);
    - построить гистограмму распределения `y_proba_best`;
    - визуально оценить баланс классов: есть ли смещение вероятностей в сторону 0 или 1.
4. Срезы по группам. Если в `X_test_sens` присутствуют `sex`, `race`, `age_group`, то дополнительно сравнить средние значения `y_proba_best` по группам - это ранний индикатор возможного смещения модели.

**Интерпретация:**
1. Равные размеры и отсутствие пропусков подтверждают корректность подготовки артефактов.
2. Сбалансированное распределение `y_proba_best` указывает на отсутствие сильной переуверенности модели.
3. Различия между группами на данном этапе - лишь диагностический сигнал, не статистически значимая мера fairness.

**Ожидаемый результат.** Построена гистограмма распределения `y_proba_best`, проверены базовые размеры и отсутствуют `NaN`. На выходе получаем подтверждение, что входные данные пригодны для расчета метрик справедливости.

In [ ]:
# --- sanity-check: размеры, NaN, распределение y_proba --- #

# проверка присутствия артефактов из A #
assert all(k in globals() for k in ["y_true_test", "y_pred_best", "y_proba_best", "X_test_sens"]), (
    "Нет алиасов от unified loader."
)

# согласованность размеров #
n = len(y_true_test)
assert len(y_pred_best) == n and len(y_proba_best) == n, "Длины y_* не совпадают"
assert len(X_test_sens) == n, "Длина X_test_sens не совпадает с y"

# NaN и допустимый диапазон #
assert np.isfinite(y_proba_best).all(), "Есть NaN/Inf в y_proba_best"
assert (y_proba_best >= 0).all() and (y_proba_best <= 1).all(), "y_proba_best вне [0, 1]"

# гистограмма вероятностей #
plt.figure()
plt.hist(y_proba_best, bins=30)
plt.xlabel("y_proba_best")
plt.ylabel("count")
plt.title("Распределение предсказанных вероятностей")
plt.tight_layout()
save_fig("hist_y_proba_best.png")
plt.show()

# базовый pos_rate по группам (t = 0.5) #
t = 0.5
y_pred_05 = (y_proba_best >= t).astype("int8")

pred_ser = pd.Series(y_pred_05, index=X_test_sens.index, name="y_pred_05")

for gcol in [c for c in ["sex", "race", "age_group"] if c in X_test_sens.columns]:
    grp = X_test_sens[gcol].fillna("NA")
    rates = pred_ser.groupby(grp).mean().sort_values(ascending=False)
    print(f"[{gcol}] selection rate @ t = 0.5:")
    print(rates, "\n")

# --- Определение чувствительных признаков и групп --- #

**Цель.** Задать список чувствительных признаков (sensitive features), по которым будет оцениваться справедливость модели, и обеспечить корректную подготовку групп для последующих метрик.

**Выбор признаков.** В соответствии с постановкой задачи *Census Income Classifier*, к чувствительным признакам относятся:
1. `sex` - бинарный (Male/Female).
2. `race` - категориальный (White/Black/Asian-Pac-Islander/Amer-Indian-Eskimo/Other).
3. `age_group` - дискретизированный возраст (например, 17-29, 30-44, 45-59, 60+), если присутствует в данных.

**Обработка данных:**
1. Проверяется наличие указанных признаков в `X_test_sens`.
2. Пропуски заменяются категорией `'NA'` для сохранения полноты групп.
3. Категориальные признаки приводятся к `category` dtype с фиксированным порядком категорий, чтобы избежать некорректного сравнения по алфавиту.
4. При необходимости малочисленные категории можно агрегировать в `'Other'`.

**Интерпретация.** Корректное определение чувствительных признаков гарантирует, что fairness-метрики (Demographic Parity, Equalized Odds и др.) будут рассчитаны для сопоставимых групп, а результаты останутся интерпретируемыми.

**Ожидаемый результат.** Сформирован DataFrame `X_test_sens` с обработанными чувствительными признаками, готовый к использованию в блоках fairness-анализа.

In [ ]:
# --- Чувствительные признаки и группы --- #

# гарантия наличия X_test_sens из unified loader #
assert "X_test_sens" in globals(), "X_test_sens должен быть из unified loader."

CAND_SENSITIVE = ["sex", "race", "age_group"]
SENSITIVE = [c for c in CAND_SENSITIVE if c in X_test_sens.columns]
print("Используем SENSITIVE:", SENSITIVE)

# осмысленный порядок для age_group #
if "age_group" in SENSITIVE:
    desired_order = ["18-25", "26-45", "46-65", "65+"]
    try:
        X_test_sens["age_group"] = pd.Categorical(
            X_test_sens["age_group"], categories=desired_order, ordered=True
        )
    except Exception as e:
        print("Не удалось задать порядок для age_group:", e)

# собираем уникальные значения по каждой чувствительной переменной #
GROUP_VALUES = {}
for col in SENSITIVE:
    vals = X_test_sens[col].dropna().unique().tolist()
    if (col == "age_group") and hasattr(X_test_sens[col], "cat"):
        vals = [v for v in X_test_sens[col].cat.categories if v in set(X_test_sens[col].dropna())]
        GROUP_VALUES[col] = list(vals)

# отчет по наличию групп и их размерам #
for col in SENSITIVE:
    print(f"\n[{col}] группы и размерности:")
    df_sizes = (
        X_test_sens[col]
        .value_counts(dropna=False)
        .rename_axis("group")
        .reset_index(name="n")
        .assign(share=lambda d: d["n"] / len(X_test_sens))
    )
    display(df_sizes)

print("\nGROUP_VALUES =", GROUP_VALUES)

# главная чувсвительная переменная #
PRIMARY_SENS = SENSITIVE[0] if SENSITIVE else None
print("PRIMARY_SENS:", PRIMARY_SENS)

# --- Базовая линия: метрики на общем пороге (0.5) --- #

**Цель.** Рассчитать ключевые метрики качества модели при фиксированном пороге `t=0.5`, который используется как базовая линия для дальнейшего анализа компромисса *качество <-> справедливость*.

**Метрики оценки:**
1. Accuracy - доля верно классифицированных наблюдений.
2. Precision - точность положительных предсказаний.
3. Recall - полнота выявления положительного класса.
4. F1-score - гармоническое среднее между Precision и Recall.
5. ROC-AUC - способность модели разделять классы независимо от порога.

**Интерпретация.** Порог `0.5` отражает "нейтральную" точку отсечения, не скорректированную под дисбаланс классов или требования fairness. Полученные значения служат:
1. Отправной точкой для поиска оптимального порога `t*`.
2. Контрольной базой при сравнении с пост-обработкой (`ThresholdOptimizer`).
3. Ориентиром для оценки влияния fairness-ограничений на качество.

**Ожидаемый результат.** Выведена таблица или словарь со значениями Accuracy, Precision, Recall, F1, ROC-AUC для `t=0.5`. Эти показатели будут использованы далее при построении пороговой кривой и Pareto-графика.

In [ ]:
# Базовые метрики на t = 0.5 #

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# бинарные предсказания при пороге 0.5 #
y_pred_05 = (y_proba_best >= 0.5).astype(int)

base_metrics = {
    "accuracy": accuracy_score(y_true_test, y_pred_05),
    "f1": f1_score(y_true_test, y_pred_05),
    "precision": precision_score(y_true_test, y_pred_05, zero_division=0),
    "roc_auc": roc_auc_score(y_true_test, y_proba_best),
    "recall": recall_score(y_true_test, y_pred_05, zero_division=0),
}

print("Базовые метрики (t = 0.5):")
display(pd.DataFrame([base_metrics]).round(4))

# --- Fairness по группам при t = 0.5 --- #

**Цель.** Оценить, насколько модель демонстрирует различия в качестве предсказаний между чувствительными группами при базовом пороге `t=0.5`.

**Используемые показатели:**
1. Selection Rate (SR) - доля объектов, классифицированных как положительные, в каждой группе.
2. True Positive Rate (TPR) и False Positive Rate (FPR) - чувствительность и ложноположительная частота по группам.
3. Precision, Recall, F1 - классические метрики производительности, рассчитанные отдельно для каждой категории чувствительного признака.
4. Demographic Parity Difference - различие в чувствительности между группами.

**Интерпретация:**
1. Различия в SR и TPR/FPR отражают потенциальное смещение модели в отношении разных подгрупп населения.
2. При значимых отклонениях (например, высокий SR у одной и низкий у другой) fairness-ограничения могут оказаться необходимыми на этапе пост-обработки.
3. Порог `0.5` здесь играет роль отправной точки, чтобы измерить масштаб возможного дисбаланса перед оптимизацией порога `t*`.

**Визуализация и результаты:**
1. Таблицы метрик по группам сохраняются в `reports/figures_03/` как CSV-файлы.
2. Для каждой метрики формируются бар-чарты (`bar_f1_by_*.png`, `bar_precision_by_*.png`, `bar_recall_by_*.png`).
3. Дополнительно строятся confusion-матрицы по группам для анализа типов ошибок.

**Ожидаемый результат.** Выведены сводные таблицы и визуализации метрик fairness при `t=0.5`. Эти результаты служат базой для выбора оптимального порога на следующем этапе.

In [ ]:
# --- Групповые fairness-метрики при t = 0.5 --- #

from fairlearn.metrics import demographic_parity_ratio, equalized_odds_difference

t = 0.5
y_pred = (y_proba_best >= t).astype("int8")

# DP/EOD @ t=0.5 #
# выбор чувствительного признака #
sf_col = globals().get("PRIMARY_SENS", None) or "sex"
sf = X_test_sens[sf_col]

dp_diff = demographic_parity_difference(y_true=y_true_test, y_pred=y_pred_05, sensitive_features=sf)
dp_ratio = demographic_parity_ratio(y_true=y_true_test, y_pred=y_pred_05, sensitive_features=sf)
eod_diff = equalized_odds_difference(y_true=y_true_test, y_pred=y_pred_05, sensitive_features=sf)

print(
    f"[fairness@0.5] {sf_col}: "
    f"DP diff={dp_diff:.4f}, "
    f"DP ratio={dp_ratio:.4f}, "
    f"EOD diff={eod_diff:.4f}"
)

_agg = {
    "settings": "t=0.5",
    "sf_col": sf_col,
    "dp_diff": dp_diff,
    "dp_ratio": dp_ratio,
    "eod_diff": eod_diff,
}
pd.DataFrame([_agg]).to_csv(FIG_DIR_03 / "fairness_overview_t05.csv", index=False)


def _prec(y_true, y_pred):
    return precision_score(y_true, y_pred, zero_division=0)


def _rec(y_true, y_pred):
    return recall_score(y_true, y_pred, zero_division=0)


def _f1(y_true, y_pred):
    return f1_score(y_true, y_pred, zero_division=0)


def _acc(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

In [ ]:
# --- score diagnostics from y_score.npy --- #

from sklearn.calibration import calibration_curve

y_score_opt, _ = A["y_score_opt"]
y_true, _ = A["y_true"]
if y_score_opt is not None:
    y_score_norm = (y_score_opt - np.min(y_score_opt)) / (np.ptp(y_score_opt) + 1e-12)
    prob_true, prob_pred = calibration_curve(y_true, y_score_norm, n_bins=10, strategy="quantile")
    df_cal = pd.DataFrame({"preb_pred": prob_pred, "prob_true": prob_true})
    df_cal.to_csv(ART_DIR / "calibration_from_score_quantile.csv", index=False)
    print("[ok] saved:", ART_DIR / "calibration_from_score_quantile.csv")
else:
    print("[skip] y_score.npy absent.")

# --- Пороговая кривая и Pareto: качество vs Demographic Parity --- #

**Цель.** Найти рабочий порог классификации `t*`, обеспечивающий оптимальный баланс между качеством модели и соблюдением принципов справедливости.

**Методика:**
1. Выполняется пороговый скан: метрики качества (Accuracy, F1) и fairness-показатели (Demographic Parity/Ratio, Equalized Odds Defference) рассчитываются на сетке значений `t ∈ [0, 1]`.
2. На основе результатов строятся кривые:
    - `accuracy_f1_vs_threshold.png` - динамика качества при изменении порога;
    - `dp_vs_threshold.png` - зависимость справедливости от порога;
    - `Pareto_f1_vs_dp.png` - компромисс между F1 и Demographic Parity.
3. Порог `t*` выбирается как точка Парето-оптимума: улучшение одной метрики без ухудшения другой становится невозможным.

**Интерпретация:**
1. При смещенной модели снижение Demographic Parity Difference обычно сопровождается потерей F1.
2. Оптимальный порог `t*` минимизирует этот компромисс, фиксируя устойчивый баланс.
3. В дальнейшем `t*` используется как основной порог для построения confusion-матриц, fairness-метрик и calibration-plots.

**Визуализация и результаты:**
1. Графики сохраняются в `reports/figures_03`.
2. Таблица с метриками по порогам (Accuracy, F1, DP, EOD) сохраняется в CSV для прозрачности аудита.
3. Отдельно сохраняется выбранный порог `t*` как артефакт (`t_star.npy`).

**Ожидаемый результат.** Построены пороговые кривые и Pareto-график. Определен и сохранен рабочий порог `t*`, отражающий оптимальный баланс между качеством и справедливостью.

In [ ]:
# --- Пороговый скан + Pareto + выбор t* + сохранение графиков --- #

import numpy as np
import pandas as pd


# заполнение NaN в sens-серии #
def _sens_fill(s: pd.Series) -> pd.Series:
    s = s.copy()
    if isinstance(s.dtype, pd.CategoricalDtype):
        if "NA" not in s.cat.categories:
            s = s.cat.add_categories(["NA"])
        return s.fillna("NA")
    else:
        return s.fillna("NA")


# какие чувствительные признаки есть в X_test_sens #
sens_cols = [c for c in ["sex", "race", "age_group"] if c in X_test_sens.columns]
if not sens_cols:
    raise RuntimeError(
        "В X_test_sens не найдено ни одного из ожидаемых чувствительных "
        "признаков: sex/race/age_group."
    )

# ensure y_true_test / y_proba_best присутствуют #
assert "y_true_test" in globals() and "y_proba_best" in globals(), (
    "Нужны y_true_test и y_proba_best из 02_modeling."
)

# выравнивание индексов по X_test_sens #
y_true = pd.Series(y_true_test, index=X_test_sens.index)
y_proba = pd.Series(y_proba_best, index=X_test_sens.index)

try:
    auc_once = roc_auc_score(y_true, y_proba)
except Exception:
    auc_once = np.nan

# Формирование пороговой таблицы #
ts = np.round(np.arange(0.05, 0.95, 0.01), 2)
rows = []
for t in ts:
    # бинаризация вероятностей на том же индексе
    y_pred_t = (y_proba >= t).astype(int)

    # базовые метрики на выровненных сериях
    m_f1 = f1_score(y_true, y_pred_t, zero_division=0)
    m_pr = precision_score(y_true, y_pred_t, zero_division=0)
    m_rc = recall_score(y_true, y_pred_t, zero_division=0)
    m_acc = accuracy_score(y_true, y_pred_t)
    m_auc = auc_once

    # DP-разница по максимуму среди имеющихся чувствительных признаков
    dp_diffs = []
    for col in sens_cols:
        s = _sens_fill(X_test_sens[col])
        grp_name = s.name or col

        # категоризация и фиксированный порядок групп
        if grp_name in GROUP_VALUES:
            s = pd.Series(
                pd.Categorical(s, categories=GROUP_VALUES[grp_name], ordered=False),
                index=y_pred_t.index,
                name=grp_name,
            )
        else:
            s = pd.Series(s, index=y_pred_t.index, name=grp_name)

        # будущий дефолт pandas: observed=True
        grp_rates = y_pred_t.groupby(s, observed=True).mean()

        # восстановление полного порядка групп
        if grp_name in GROUP_VALUES:
            grp_rates = grp_rates.reindex(GROUP_VALUES[grp_name])

        # расчет disparate impact/difference
        dp = float(grp_rates.max() - grp_rates.min())
        dp_diffs.append(dp)

    # если все NaN — вернётся np.nan #
    dp_diff_max = float(np.nanmax(dp_diffs)) if len(dp_diffs) else np.nan

    # EOD #
    eod_diffs = []
    try:
        mask_pos = y_true == 1
        for col in sens_cols:
            s = _sens_fill(X_test_sens[col])
            grp_name = s.name or col

            if grp_name in GROUP_VALUES:
                s = pd.Series(
                    pd.Categorical(s, categories=GROUP_VALUES[grp_name], ordered=False),
                    index=y_true.index,
                    name=grp_name,
                )
                groups_iter = GROUP_VALUES[grp_name]
            else:
                s = pd.Series(s, index=y_true.index, name=grp_name)
                groups_iter = pd.unique(s)

            tprs = []
            for g in groups_iter:
                m = (s == g) & mask_pos
                if m.sum() > 0:
                    tprs.append((y_pred_t[m] == 1).mean())
            if len(tprs) > 1:
                eod_diffs.append(float(np.max(tprs) - np.min(tprs)))
        eod_diff_max = float(np.nanmax(eod_diffs)) if eod_diffs else np.nan
    except Exception:
        eod_diff_max = np.nan

    rows.append(
        {
            "t": t,
            "f1": m_f1,
            "precision": m_pr,
            "recall": m_rc,
            "accuracy": m_acc,
            "auc": m_auc,
            "dp_diff_max": dp_diff_max,
            "eod_diff_max": eod_diff_max,
        }
    )

scan_df = pd.DataFrame(rows)

# экспорт csv #
ART_DIR = globals().get("ART_DIR", Path("data") / "artifacts")
ART_DIR.mkdir(parents=True, exist_ok=True)
out_csv = ART_DIR / "fairness_threshold_scan.csv"
scan_df.to_csv(out_csv, index=False)
print(f"Saved fairness_threshold_scan.csv -> {out_csv}")

# копия в reports #
REPORTS_DIR = globals().get("REPORTS_DIR", Path("data") / "reports")
FIG_DIR_03 = REPORTS_DIR / "figures_03"
FIG_DIR_03.mkdir(parents=True, exist_ok=True)
out_csv_rep = FIG_DIR_03 / "fairness_threshold_scan.csv"
scan_df.to_csv(out_csv_rep, index=False)
print(f"Copied to reports -> {out_csv_rep}")

# Pareto: F1 vs DP #
fig, ax = plt.subplots(figsize=(6, 5))
ax.scatter(scan_df["dp_diff_max"], scan_df["f1"], s=14, alpha=0.85)
ax.set_xlabel("Demographic Parity (max group diff)")
ax.set_ylabel("F1")
ax.set_title("Pareto: F1 vs DP")

# выделение лучшей по F1 точки #
best_idx = int(scan_df["f1"].idxmax())
ax.scatter(
    [scan_df.loc[best_idx, "dp_diff_max"]], [scan_df.loc[best_idx, "f1"]], s=60, edgecolors="k"
)
for k in ["dp_diff_max", "f1", "t"]:
    ax.annotate(
        f"{k}={scan_df.loc[best_idx, k]:.3f}",
        (scan_df.loc[best_idx, "dp_diff_max"], scan_df.loc[best_idx, "f1"]),
        xytext=(10, 10),
        textcoords="offset points",
        fontsize=8,
    )
save_fig("Pareto_f1_vs_dp.png", fig)

# метрики vs порог #
fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(scan_df["t"], scan_df["accuracy"], label="accuracy")
ax.plot(scan_df["t"], scan_df["f1"], label="f1", linestyle="--")
ax.set_xlabel("threshold t")
ax.set_ylabel("score")
ax.set_title("Accuracy & F1 vs threshold")
ax.legend()
save_fig("accuracy_f1_vs_threshold.png", fig)

fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(scan_df["t"], scan_df["dp_diff_max"])
ax.set_xlabel("threshold t")
ax.set_ylabel("DP max diff")
ax.set_title("DP (max group diff) vs threshold")
save_fig("dp_vs_threshold.png", fig)

# EOD #
if scan_df["eod_diff_max"].notna().any():
    fig, ax = plt.subplots(figsize=(7, 5))
    ax.plot(scan_df["t"], scan_df["eod_diff_max"])
    ax.set_xlabel("threshold t")
    ax.set_ylabel("EOD max diff")
    ax.set_title("EOD (max group diff) vs threshold")
    save_fig("eod_vs_threshold.png", fig)

# Pareto-front: F1 vs DP #
scan_df["dp_abs"] = scan_df["dp_diff_max"].abs()

pareto_idx = []
for i, r in scan_df.iterrows():
    dominated = (
        (scan_df["f1"] >= r["f1"])
        & (scan_df["dp_abs"] <= r["dp_abs"])
        & ((scan_df["f1"] > r["f1"]) | (scan_df["dp_abs"] < r["dp_abs"]))
    ).any()
    if not dominated:
        pareto_idx.append(i)
pareto_df = scan_df.loc[pareto_idx]

# критерий выбора: минимальный |DP|, при равенстве - максимальный F1 #
t_star = float(pareto_df.sort_values(["dp_abs", "f1"], ascending=[True, False]).iloc[0]["t"])

# сохранить t* как артефакт #
np.save(ART_DIR / "t_star.npy", np.array([t_star], dtype=float))

In [ ]:
# --- Compare metrics: t* vs t-0.5 --- #

assert "t_star" in globals()
y_true, _ = A["y_true"]
y_proba_best, _ = A["y_proba"]
y_pred_050_opt, _ = A["y_pred_050_opt"]


def _metrics(y_true, y_pred, y_score=None):
    out = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }
    if y_score is not None:
        try:
            out["roc_auc"] = roc_auc_score(y_true, y_score)
        except Exception:
            pass
    return out


# t* #
y_pred_star = (y_proba_best >= float(t_star)).astype(int)
m_star = _metrics(y_true, y_pred_star, y_proba_best)

# t=0.5 #
if y_pred_050_opt is not None:
    m_050 = _metrics(y_true, y_pred_050_opt, y_proba_best)
    comp = pd.DataFrame([m_050, m_star], index=["t=0.5", "t*"])
    display(comp)
    comp.to_csv(ART_DIR / "metrics_t050_vs_tstar.csv", index=True)
    print("[ok] saved:", ART_DIR / "metrics_050_vs_tstar.csv")
else:
    print("[skip] y_pred_050.npy absent.")

In [ ]:
# --- Group metrics @ t* -> CSV --- #

# входные объекты: #
# y_true : вектор истинных меток #
# y_proba_best : вероятности положительного класса #
# t_star : выбранный порог #
# X_test_sensitive : DataFrame с чувствительными признаками #

from sklearn.metrics import precision_recall_fscore_support

assert "t_star" in globals(), "t_star не найден"
assert "y_proba_best" in globals(), "y_proba_best не найден"
assert "y_true" in globals(), "y_true не найден"
assert "X_test_sensitive" in globals(), "X_test_sensitive не найден"

# каталоги вывода #
ART_DIR = globals().get("ART_DIR", Path("data") / "artifacts")
FIG_DIR_03 = globals().get("REPORTS_DIR", Path("data") / "reports") / "figures_03"

ART_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR_03.mkdir(parents=True, exist_ok=True)

# предсказания на пороге t* #
y_pred_star = (y_proba_best >= float(t_star)).astype(int)

rows_metrics = []
rows_selrate = []

for col in X_test_sensitive.columns:
    s = X_test_sensitive[col]
    # отброс пропусков из группировки
    for g in sorted(s.dropna().unique()):
        m = (s == g).values
        n = int(m.sum())
        if n == 0:
            continue
        yt = y_true[m]
        yp = y_pred_star[m]

        # metrics
        p, r, f1, _ = precision_recall_fscore_support(yt, yp, average="binary", zero_division=0)
        acc = accuracy_score(yt, yp)

        rows_metrics.append(
            {
                "threshold": float(t_star),
                "group_col": col,
                "group": g,
                "n": n,
                "precision": float(p),
                "recall": float(r),
                "f1": float(f1),
                "accuracy": float(acc),
            }
        )

        # selection rate
        sel = float(yp.mean()) if n > 0 else 0.0
        rows_selrate.append(
            {
                "threshold": float(t_star),
                "group_col": col,
                "group": g,
                "n": n,
                "selection_rate": sel,
            }
        )

df_metrics = pd.DataFrame(
    rows_metrics,
    columns=["threshold", "group_col", "group", "n", "precision", "recall", "f1", "accuracy"],
)
df_selrate = pd.DataFrame(
    rows_selrate, columns=["threshold", "group_col", "group", "n", "selection_rate"]
)

# сохранение в отчеты и дубликат в артефакты #
p1 = FIG_DIR_03 / "group_metrics_t_star.csv"
p2 = FIG_DIR_03 / "selection_rates_t_star.csv"
a1 = ART_DIR / "group_metrics_t_star.csv"
a2 = ART_DIR / "selection_rates_t_star.csv"

df_metrics.to_csv(p1, index=False)
df_selrate.to_csv(p2, index=False)
df_metrics.to_csv(a1, index=False)
df_selrate.to_csv(a2, index=False)

print(f"[ok] Saved: {p1}")
print(f"[ok] Saved: {p2}")
print(f"[ok] Saved: {a1}")
print(f"[ok] Saved: {a2}")

# --- Post-Processing: ThresholdOptimizer (DemographicParity / EqualizedOdds) --- #

**Цель.** Применить пост-обработку для корректировки решений модели с учетом fairness-ограничений и оценить влияние этой процедуры на качество и справедливость.

**Подход:**
1. Используется метод `ThresholdOptimizer` из библиотеки fairlearn, позволяющий адаптировать порог классификации отдельно для каждой подгруппы чувствительного признака.
2. Рассматриваются два типа ограничений:
    - Demogrpahic Parity - выравнивание долей положительных исходов между группами;
    - Equalize Odds - выравнивание чувствительности (TPR) и специфичности (FPR) между группами.

**Методика сравнения:**
1. Выполнить оптимизацию отдельно для каждого ограничения, используя обученную модель как черный ящик (`estimator='prefitted'`).

In [ ]:
# --- Post‑processing: ThresholdOptimizer (DP / EqOdds) --- #

from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference

warnings.filterwarnings("ignore", category=UserWarning, message=".*sensitive features not unique.*")


# заполнение NaN в чувствительных признаках #
def _sens_fill(s):
    s = s.copy()
    if isinstance(s.dtype, pd.CategoricalDtype):
        if "NA" not in s.cat.categories:
            s = s.cat.add_categories(["NA"])
        return s.fillna("NA")
    else:
        return s.fillna("NA")


# выбор доступного чувствительного признака #
sens_cols = [c for c in ["sex", "race", "age_group"] if c in X_test_sens.columns]
if not sens_cols:
    print("Нет сенситивных признаков - пропуск ThresholdOptimizer")
else:
    gcol = "sex" if "sex" in X_test_sens.columns else sens_cols[0]
    sf = _sens_fill(X_test_sens[gcol])

    class _ScoresEstimator:
        """Суррогат-классификатор, который отдает уже посчитанные вероятности."""

        def __init__(self, scores):
            self.scores = np.asarray(scores)
            # чтобы check_fitted не ругался при prefit=True
            self.fitted_ = True

        def get_params(self, deep=True):
            return {"scores": self.scores}

        def set_params(self, **params):
            # запоминание размерности
            return self

        def fit(self, X, y):
            self.fitted_ = True
            # запоминание размерности
            return self

        def predict_proba(self, X):
            # X используется только для совместимости сигнатуры;
            # возвращаем заранее посчитанные вероятности
            p = self.scores
            return np.column_stack([1.0 - p, p])

    # создаем surrogate-модель и "признаки" как индексы #
    X_idx = np.arange(len(y_true_test)).reshape(-1, 1)
    base_est = _ScoresEstimator(y_proba_best)

    # ThresholdOptimizer под демографический паритет #
    postproc = ThresholdOptimizer(
        estimator=base_est,
        constraints="demographic_parity",
        predict_method="predict_proba",
        prefit=True,
    )
    # fit/predict на одних и тех же X_idx #
    postproc.fit(X=X_idx, y=y_true_test, sensitive_features=sf)
    y_pred_dp = postproc.predict(X=X_idx, sensitive_features=sf).astype("int8")

    # метрики после пост-обработки #
    def _prec(y_true, y_pred):
        return precision_score(y_true, y_pred, zero_division=0)

    def _rec(y_true, y_pred):
        return recall_score(y_true, y_pred, zero_division=0)

    def _f1(y_true, y_pred):
        return f1_score(y_true, y_pred, zero_division=0)

    dp = demographic_parity_difference(y_true=y_true_test, y_pred=y_pred_dp, sensitive_features=sf)
    eq = equalized_odds_difference(y_true=y_true_test, y_pred=y_pred_dp, sensitive_features=sf)

    print(
        f"[ThresholdOptimizer @ {gcol}] "
        f"acc={accuracy_score(y_true_test, y_pred_dp):.3f} "
        f"f1={_f1(y_true_test, y_pred_dp):.3f} "
        f"prec={_prec(y_true_test, y_pred_dp):.3f} "
        f"rec={_rec(y_true_test, y_pred_dp):.3f} "
        f"| DP diff={dp:+.3f} EqOdds diff={eq:+.3f}"
    )

# сводная таблица метрик: baseline vs t* ThresholdOptimizer #
rows = []


def _metrics_row(tag, y_pred_local, sf_local):
    return {
        "settings": tag,
        "accuracy": accuracy_score(y_true_test, y_pred_local),
        "precision": precision_score(y_true_test, y_pred_local, zero_division=0),
        "recall": recall_score(y_true_test, y_pred_local, zero_division=0),
        "f1": f1_score(y_true_test, y_pred_local, zero_division=0),
        "dp_diff": demographic_parity_difference(
            y_true=y_true_test, y_pred=y_pred_local, sensitive_features=sf_local
        ),
        "eod_diff": equalized_odds_difference(
            y_true=y_true_test, y_pred=y_pred_local, sensitive_features=sf_local
        ),
    }


# baseline t=0.5 #
y_pred_05 = (y_proba_best >= 0.5).astype(int)
rows.append(_metrics_row("baseline_t0.5", y_pred_05, sf))

# baseline t* #
if "t_star" in globals():
    y_pred_star = (y_proba_best >= t_star).astype(int)
    rows.append(_metrics_row("baseline_t*", y_pred_star, sf))

# post-processing DP #
rows.append(_metrics_row("ThresholdOptimizer_DP", y_pred_dp, sf))

postproc_df = pd.DataFrame(rows)
postproc_df.to_csv(FIG_DIR_03 / "metrics_postprocessing.csv", index=False)
print("[postproc] saved:", FIG_DIR_03 / "metrics_postprocessing.csv")

# --- Калибровка вероятностей по группам --- #

**Цель.** Оценить согласованность вероятностных предсказаний с эмпирическими частотами положительного исхода в разных чувствительных группах.

**Методика:**
1. Для каждой доступной группы (`sex`, `race`, `age_group`) строим кривые надежности (reliability curves) по 10 равным бинам вероятностей.
2. Считаем Expected Calibration Error (ECE) по 10 бинам.
3. Игнорируем слишком малые подгруппы при построении графиков, задавая `min_count=50`.

**Результаты:**
1. Файлы графиков: `calibration_{group}.png` в `reports/figures_03/`.
2. Сводная таблица ECE: `calibration_ece_by_group.png` в `reports/figures_03/`.

**Интерпретация:**
1. Линия \(y=x\) соответствует идеальной калибровке. Систематические отклонения указывают на пере/недоуверенность модели в конкретной группе.
2. ECE суммирует средневзвешенную величину отклонений и служит компактным числовым индикатором калибровки по группам.

In [ ]:
# --- Calibration plots by groups + сохранение --- #

from sklearn.calibration import calibration_curve


def plot_reliability_by_groups(
    y_true, y_proba, group_series, n_bins=10, min_count=50, title="", save_name=None
):
    groups = group_series.value_counts().index.tolist()
    kept = []
    fig, ax = plt.subplots(figsize=(6, 6))
    for g in groups:
        mask = (group_series == g).to_numpy()
        if mask.sum() < min_count:
            continue
        prob_true, prob_pred = calibration_curve(
            y_true[mask], y_proba[mask], n_bins=n_bins, strategy="uniform"
        )
        ax.plot(prob_pred, prob_true, label=f"{g} (n={mask.sum()})")
        kept.append(g)
    ax.plot([0, 1], [0, 1], "--", linewidth=1)
    ax.set_xlabel("Predicted probability")
    ax.set_ylabel("Empirical positive rate")
    ax.set_title(title or f"Calibration by {group_series.name}")
    if kept:
        ax.legend()
    if save_name:
        save_fig(save_name, fig)


def compute_ece(y_true, y_proba, n_bins=10):
    # равномерная бинингация по предсказанной вероятности #
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    idx = np.digitize(y_proba, bins) - 1
    ece = 0.0
    N = len(y_true)
    for b in range(n_bins):
        mask = idx == b
        nb = mask.sum()
        if nb == 0:
            continue
        acc_b = y_true[mask].mean()
        conf_b = y_proba[mask].mean()
        ece += (nb / N) * abs(acc_b - conf_b)
    return float(ece)


# защита наличия данных #
assert all(k in globals() for k in ["y_true_test", "y_proba_best", "X_test_sens"])

# вызовы на все доступные чувствительные признаки #
for col in [c for c in ["sex", "race", "age_group"] if c in X_test_sens.columns]:
    plot_reliability_by_groups(
        y_true=y_true_test,
        y_proba=y_proba_best,
        group_series=X_test_sens[col],
        n_bins=10,
        min_count=50,
        title=f"Calibration by {col}",
        save_name=f"calibration_{col}.png",
    )

# ECE по группам и сохранение CSV #
ece_rows = []
for col in [c for c in ["sex", "race", "age_group"] if c in X_test_sens.columns]:
    s = X_test_sens[col]
    # по каждой категории в признаке col
    for g in s.dropna().unique():
        m = (s == g).to_numpy()
        if m.sum() == 0:
            continue
        ece = compute_ece(y_true=y_true_test[m], y_proba=y_proba_best[m], n_bins=10)
        ece_rows.append({"group_col": col, "group": str(g), "n": int(m.sum()), "ece10": ece})

if ece_rows:
    pd.DataFrame(ece_rows).sort_values(["group_col", "group"]).to_csv(
        FIG_DIR_03 / "calibration_ece_by_group.csv", index=False
    )

In [ ]:
# --- Group metrics bar charts + сохранение --- #

assert "t_star" in globals(), "Ожидается выбранный порог t_star"
assert all(k in globals() for k in ["y_true_test", "y_proba_best", "X_test_sens"])
y_pred_star = (y_proba_best >= t_star).astype(int)


def _bar_metric_by_group(
    y_true, y_pred, group_series, metric_fn, metric_name: str, min_count=50, save_name=""
):
    recs = []
    for g, idx in (
        group_series.reset_index(drop=True)
        .groupby(group_series.reset_index(drop=True))
        .groups.items()
    ):
        idx = np.array(idx, dtype=int)
        if len(idx) < min_count:
            continue
        val = metric_fn(y_true[idx], y_pred[idx])
        recs.append({"group": str(g), metric_name: float(val), "n": int(len(idx))})
    if not recs:
        print(f"[fairness] skip {metric_name}: all groups < min_count")
        return
    df = pd.DataFrame(recs).sort_values(metric_name, ascending=False)

    fig, ax = plt.subplots(figsize=(7, 4))
    ax.bar(df["group"], df[metric_name])
    ax.set_title(f"{metric_name} by {group_series.name}")
    ax.set_ylabel(metric_name)
    ax.set_xlabel(group_series.name)
    for i, v in enumerate(df[metric_name].values):
        ax.text(i, v, f"{v:.2f}", ha="center", va="bottom", fontsize=8)
    save_fig(save_name, fig)


for col in [c for c in ["sex", "race", "age_group"] if c in X_test_sens.columns]:
    gs = X_test_sens[col]
    _bar_metric_by_group(
        y_true_test,
        y_pred_star,
        gs,
        precision_score,
        "precision",
        save_name=f"bar_precision_by_{col}.png",
    )
    _bar_metric_by_group(
        y_true_test, y_pred_star, gs, recall_score, "recall", save_name=f"bar_recall_by_{col}.png"
    )
    _bar_metric_by_group(
        y_true_test, y_pred_star, gs, f1_score, "f1", save_name=f"bar_f1_by_{col}.png"
    )

In [ ]:
# --- Confusion matrices by group + сохранение --- #

import seaborn as sns
from sklearn.metrics import confusion_matrix

assert "t_star" in globals()
assert all(k in globals() for k in ["y_true_test", "y_proba_best", "X_test_sens"])
y_pred_star = (y_proba_best >= t_star).astype(int)


def _plot_cm(y_true, y_pred, title=""):
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    fig, ax = plt.subplots(figsize=(4.5, 4))
    sns.heatmap(cm, annot=True, fmt="d", cbar=False, ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title(title)
    ax.set_xticklabels(["0", "1"])
    ax.set_yticklabels(["0", "1"])
    return fig


for col in [c for c in ["sex", "race", "age_group"] if c in X_test_sens.columns]:
    s = X_test_sens[col]
    for g in s.dropna().unique():
        mask = (s == g).to_numpy()
        if mask.sum() < 50:
            continue
        fig = _plot_cm(
            y_true_test[mask], y_pred_star[mask], title=f"CM: {col}={g} (n={mask.sum()})"
        )
        safe_g = str(g).replace("/", "-").replace("\\", "-").replace(" ", "_")
        save_fig(f"cm_{col}_{safe_g}.png", fig)

In [ ]:
# --- Baselines visuals from figures_02 --- #

figs02, dir02 = A["figures02"]
if figs02:
    print(f"[info] figures_02: {dir02} | count={len(figs02)}")
    for p in figs02[:6]:
        print(" -", p.name)
else:
    print("[skip] figures_02 not found.")

# --- Explainability: глобально (SHAP / альтернативы) --- #

**Цель.** Понять, какие признаки вносят наибольший вклад в предсказания модели и нет ли косвенных зависимостей от чувствительных характеристик.

**Методика.** Для финальной модели (LightGBM/XGBoost) использован `shap.TreeExplainer`, позволяющий оценить вклад каждого признака в прогноз вероятности класса `>50k`. SHAP обеспечивает консистентную интерпретацию влияния признаков, что делает его предпочтительным методом для объяснимости бустинговых моделей.

**Результаты глобального анализа:**
1. Вклад ключевых социально-экономических признаков (например, `marital-status_Married-civ-spouse`, `education-num`, `capital-gain`, `hours-per-week`, `age`) значительно выше остальных.
2. Чувствительные переменные (`sex`, `race`, `age_group`) имеют низкое среднее влияние, что говорит об отсутствии прямого использования этих признаков моделью.
3. Однако частичные зависимости (через прокси-признаки вроде `occupation`, `marital-status`) могут опосредованно отражать различия между группами.

**Вывод.** SHAP подтверждает, что модель в целом фокусируется на экономических и демографических характерисках, релевантных доходу, а не на чувствительных признаках напрямую. Это согласуется с результатами fairness-оценки, где наблюдались умеренные, но не критичные различия по группам.

In [ ]:
# --- Explainability input selection via unified loader --- #

assert "A" in globals(), "Выполните unified loader"
model, _ = A["model"]
XS, _ = A["X_test_enc"]
feature_names, _ = A["feature_names"]
test_groups, _ = A["test_groups"]

# прямое использование загруженных артефактов #
if XS is not None and feature_names is not None:
    clf_for_shap = getattr(model, "best_estimator_", model)
    msg_ok = (
        f"[ok] XS: {getattr(XS, 'shape', None)}, "
        f"features: {len(feature_names)}, "
        f"model: {type(clf_for_shap).__name__}"
    )
    print(msg_ok)
else:
    # fallback только если что-то отсутствует
    warn_msg = (
        "[warn] Нет XS или feature_names из артефактов. "
        "Включено...fallback-восстановление "
        "(отключено по политике unified loader)."
    )
    print(warn_msg)
    raise RuntimeError(
        "Отсутствуют необходимые артефакты для explainability: X_test_enc и/или feature_names."
    )

In [ ]:
# --- Dependence-plots для топ-фич --- #

import re

import scipy.sparse as sp

rng = np.random.default_rng(42)

# поиск артефактов во всех типичных папках проекта #
ART_DIRS = [
    Path("data") / "artifacts",
    Path("data") / "models",
    Path("notebooks") / "artifacts",
    Path("notebooks") / "models",
]


def _first_exists_any(names):
    names_lower = [n.lower() for n in names]
    for d in ART_DIRS:
        if not d.exists():
            continue
        for p in d.iterdir():
            if p.is_file() and p.name.lower() in names_lower:
                return p
    # рекурсивный fallback
    roots = [Path(".").resolve()]
    roots += list(roots[0].parents)[:3]
    for root in roots:
        for p in root.rglob("*"):
            try:
                if p.is_file() and p.name.lower() in names_lower:
                    return p
            except PermissionError:
                continue
    return None


# модель #
clf = None
p_model = _first_exists_any(["lgb_best.joblib", "LGBM_best.joblib", "model_best.joblib"])
if p_model is not None:
    model, model_path = A["model"]
    obj = model
    # если это pipeline — достанем из него clf
    if hasattr(obj, "named_steps"):
        clf = obj.named_steps.get("clf", None) or obj
    else:
        clf = obj

# данные: X_test_enc и feature_names #
X_test_enc, X_test_enc_path = A["X_test_enc"]
XS = X_test_enc

# загрузка feature_names из артефактов #
feature_names, feature_names_path = A["feature_names"]

# защита от отсутствия данных #
if XS is None:
    print("Нет данных XS/X_test_enc: пропуск dependence-plots.")
    raise SystemExit

# выравнивание feature_names под текущую матрицу XS #
ncols = XS.shape[1]

# попытка взять имена из модели (если не дефолтные f0..fN) #
try:
    if (
        "clf" in globals()
        and clf is not None
        and hasattr(clf, "booster_")
        and clf.booster_ is not None
    ):
        fnm = list(clf.booster_.feature_name())
        all_fnums = all(isinstance(x, str) and re.fullmatch(r"f\d+", x) for x in fnm)
        if isinstance(fnm, list) and len(fnm) == ncols and not all_fnums:
            feature_names = [str(x) for x in fnm]
            print("[info] feature_names взяты из модели (booster).")
except Exception:
    pass


# гарантия совпадения длины: обрезать/дополнить при необходимости #
def _ensure_len(cols, n):
    lst = list(cols) if isinstance(cols | list | np.ndarray | pd.Index) else []
    if len(lst) == n:
        return [str(c) for c in lst]
    if len(lst) > n:
        print(f"[warn] feature_names длиннее ({len(lst)}) ncols={n}. Обрезаем.")
        return [str(c) for c in lst[:n]]
    print(f"[warn] feature_names короче ({len(lst)}) ncols={n}. Дополняем f{len(lst)}..f{n - 1}.")
    return [str(c) for c in (lst + [f"f{i}" for i in range(len(lst), n)])]


if feature_names is None or (
    isinstance(feature_names, list | np.ndarray | pd.Index) and len(feature_names) != ncols
):
    feature_names = _ensure_len(feature_names, ncols)

if XS is not None and not isinstance(XS, pd.DataFrame):
    if sp.issparse(XS):
        XS = pd.DataFrame.sparse.from_spmatrix(
            XS, columns=(feature_names if feature_names is not None else None)
        )
    else:
        XS = pd.DataFrame(XS, columns=(feature_names if feature_names is not None else None))

# выделение классификатора из pipeline при необходимости #
clf_for_shap = None
if "clf" in globals() and clf is not None and hasattr(clf, "predict_proba"):
    clf_for_shap = clf
else:
    # безопасная проверка pipeline
    g = globals()
    _pipe = g.get("pipe", None)

    if (_pipe is not None) and hasattr(_pipe, "named_steps"):
        steps = getattr(_pipe, "named_steps", {})
        clf_for_shap = steps.get("clf")
        if clf_for_shap is None:
            for _, step in steps.items():
                if hasattr(step, "predict_proba") or hasattr(step, "predict"):
                    clf_for_shap = step
                    break

# единая проверка и подвыборка #
if clf_for_shap is None or XS is None:
    print("Нет пригодной модели/данных для SHAP: пропуск dependence-plots.")
else:
    # подвыборка
    idx = np.arange(len(XS))
    if len(idx) > 5000:
        idx = rng.choice(idx, size=5000, replace=False)
    XS_sub = XS.iloc[idx] if hasattr(XS, "iloc") else XS[idx, :]

    expl = shap.TreeExplainer(clf_for_shap)
    shap_vals = expl.shap_values(XS_sub)

    # бинарный класс -> берем вклад позитивного класса
    if isinstance(shap_vals, list):
        try:
            classes_ = getattr(clf_for_shap, "classes_", [0, 1])
            pos_idx = list(classes_).index(1)
        except Exception:
            pos_idx = 1 if len(shap_vals) > 1 else 0
        shap_vals = shap_vals[pos_idx]

    # top-k
    topk = 10

    # beeswarm -> файл
    shap.summary_plot(
        shap_vals,
        XS_sub,
        feature_names=(feature_names if feature_names is not None else None),
        show=False,
        max_display=topk,
    )
    plt.tight_layout()
    save_fig("shap_summary_top10.png", None)
    plt.close()

    # bar -> файл
    shap.summary_plot(
        shap_vals,
        XS_sub,
        feature_names=(feature_names if feature_names is not None else None),
        plot_type="bar",
        show=False,
        max_display=topk,
    )
    plt.tight_layout()
    save_fig("shap_bar_top10.png", None)
    plt.close()

In [ ]:
# --- Explainability: локально (кейсы TP/FP/FN) --- #

assert all(k in globals() for k in ["y_true_test", "y_proba_best", "X_test_sens"])
thr = globals().get("t_star", 0.5)
y_pred_thr = (y_proba_best >= thr).astype("int8")

TP_idx = np.where((y_true_test == 1) & (y_pred_thr == 1))[0]
FP_idx = np.where((y_true_test == 0) & (y_pred_thr == 1))[0]
FN_idx = np.where((y_true_test == 1) & (y_pred_thr == 0))[0]

# только артефакты из unified loader #
XS, _ = A["X_test_enc"]
feature_names, _ = A["feature_names"]
model, _ = A["model"]

# извлечение классификатора из best_estimator_/Pipeline #
obj = getattr(model, "best_estimator_", model)
if hasattr(obj, "named_steps"):
    clf_for_shap = obj.named_steps.get("clf", None)
    if clf_for_shap is None:
        for step in obj.named_steps.values():
            if hasattr(step, "predict_proba") or hasattr(step, "predict"):
                clf_for_shap = step
                break
else:
    clf_for_shap = obj

if clf_for_shap is None:
    raise RuntimeError("Не удалось извлечь классификатор из Pipeline для SHAP.")

if XS is None and feature_names is None:
    raise RuntimeError(
        "Нет X_test_enc или feature_names из артефактов; не выполняем explainability."
    )

# построение SHAP-графиков #
expl = shap.TreeExplainer(clf_for_shap)


def _pick_some(arr, k=3):
    if len(arr) == 0:
        return []
    rng = np.random.default_rng(42)
    return arr if len(arr) <= k else rng.choice(arr, size=k, replace=False).tolist()


cases = [
    ("TP", _pick_some(TP_idx, 3)),
    ("FP", _pick_some(FP_idx, 3)),
    ("FN", _pick_some(FN_idx, 3)),
]

for tag, idxs in cases:
    if not idxs:
        print(f"{tag}: нет кейсов - пропуск.")
        continue
    for i in idxs:
        if hasattr(XS, "iloc"):
            row = XS.iloc[[i]]
        elif sp.issparse(XS):
            row = XS[i]
        else:
            row = XS[i : i + 1]

        sv = expl.shap_values(row)

        # выбор класс и разжитие до 2D
        sv_arr = sv[1] if isinstance(sv, list) else sv
        if getattr(sv_arr, "ndim", 1) == 1:
            sv_arr = sv_arr.reshape(1, -1)

        # LightGBM: последний столбец = base value -> вынести и обрезать
        if sv_arr.shape[1] == row.shape[1] + 1:
            base_val = sv_arr[0, -1]
            sv_arr = sv_arr[:, :-1]
        else:
            ev = expl.expected_value
            base_val = ev[1] if isinstance(ev, list | np.ndarray) else ev

        # признаки как 1D
        if hasattr(row, "values"):  # DataFrame
            feats_1d = row.values.reshape(-1)
        else:  # np/sparse
            feats_1d = np.asarray(row).reshape(-1)

        fig = shap.force_plot(
            base_value=base_val,
            shap_values=sv_arr.reshape(-1),
            features=feats_1d,
            feature_names=feature_names,
            matplotlib=True,
            show=False,
        )
        plt.suptitle(f"{tag} case idx={i} @ t={thr:.2f}")
        plt.tight_layout()
        plt.show(fig)

# --- Риски, ограничения, рекомендации --- #

**Качество и справедливость.** Финальная модель демонстрирует высокий уровень предсказательной способности (ROC-AUC около 0.9) при умеренном компромиссе между точностью и справедливостью. Без коррекции fairness наблюдается дисбаланс в доле положительных предсказанйи между полами, однако применение пост-обработки (ThresholdOptimizer с демографическим паритетом или равными шансами) значительно снижает разрыв. Это сопровождается небольшим падением F1-метрики - типичный эффект балансировки fairness и accuracy.

**Калибровка и интерпретируемость.** Для крупных групп модель хорошо откалибрована: вероятности адекватно отражают реальные частоты положительного исхода. Для малых групп (по возрасту или редким рассовым категориям) погрешность выше, что объясняется ограниченным числом наблюдений. Глобальный SHAP-анализ подтверждает, что модель опирается прежде всего на социально-экономические признаки (`marital-status`, `education-num`, `capital-gain`, `hours-per-week`, `age`), а чувствительные признаки имеют минимальное прямое влияние.

**Риски и направления улучшения:**
1. Небольшие группы дают нестабильные fairness-метрики и ухудление калибровки.
2. Возможные прокси-факторы (например, семейное положение или профессия) могут частично кодировать чувствительные характеристики.
3. Усиление fairness-регуляции может привести к заметной потере точности.

**Рекомендации:**
1. Для практического использования целесообразно применять модель с базовым порогом `t=0.5` или оптимальным `t*`.
2. В задачах с критичными требованиями к справедливости использовать ThresholdOptimizer с контролем демографического паритета.
3. Проверить влияние или агрегации прокси-фичей на устойчивость fairness-метрик.
4. При необходимости провести перекалибровку вероятностей (Platt или Isotonic) и расширить данные по малым подгруппам.

In [ ]:
# --- Экспорт артефактов из 03_fairness_and_explainability.ipynb --- #

ART_DIR = globals().get("ART_DIR", Path("data") / "artifacts")
REPORTS_DIR = globals().get("REPORTS_DIR", Path("data") / "reports")

ART_DIR.mkdir(parents=True, exist_ok=True)
(REPORTS_DIR / "figures_03").mkdir(parents=True, exist_ok=True)

# экспорт сканирования порогов #
scan_obj = globals().get("scan_df", None)
try:
    if isinstance(scan_obj, pd.DataFrame) and len(scan_obj) > 0:
        out_csv = ART_DIR / "fairness_threshold_scan.csv"
        scan_obj.to_csv(out_csv, index=False)
        print(f"Saved fairness_threshold_scan.csv -> {out_csv}")

        # копия в reports для удобства просмотра
        out_csv_rep = REPORTS_DIR / "figures_03" / "fairness_threshold_scan.csv"
        try:
            scan_obj.to_csv(out_csv_rep, index=False)
            print(f"Copied to reports -> {out_csv_rep}")
        except Exception as e:
            print("[warn] copy to reports failed:", e)
    else:
        print("[info] scan_df отсутствует или пуст - экспорт пропущен.")
except Exception as e:
    print("[warn] scan_df export:", e)

print("Экспорт артефактов из 03_fairness_and_explainability.ipynb завершен.")

In [ ]:
# --- Контроль готовности --- #

# базовые директории #
ROOT = globals().get("ROOT", Path("."))
ART_DIR = globals().get("ART_DIR", ROOT / "data" / "artifacts")
REPORTS_DIR = globals().get("REPORTS_DIR", ROOT / "data" / "reports")
FIG_DIR_03 = REPORTS_DIR / "figures_03"

# проверка наличия ключевых директорий #
assert ART_DIR.exists(), f"Нет папки артефактов: {ART_DIR}"
FIG_DIR_03.mkdir(parents=True, exist_ok=True)

# обязательные объекты в памяти #
need = ["feature_names", "X_test_enc", "sensitive", "y_true", "y_proba", "y_pred"]
miss = [k for k in need if (k not in A) or (A.get(k, (None, None))[0] is None)]
if miss:
    raise RuntimeError(f"Не загружены артефакты {miss}")

# ожидаемые файлы и паттерны #
critical = [
    FIG_DIR_03 / "fairness_threshold_scan.csv",
    ART_DIR / "fairness_threshold_scan.csv",
    FIG_DIR_03 / "shap_summary_top10.png",
    FIG_DIR_03 / "shap_bar_top10.png",
    FIG_DIR_03 / "group_metrics_t_star.csv",
    FIG_DIR_03 / "selection_rates_t_star.csv",
]

patterns = [
    "Pareto_f1_vs_dp.png",
    "accuracy_f1_vs_threshold.png",
    "dp_vs_threshold.png",
    "eod_vs_threshold.png",
    "calibration_*.png",
    "bar_precision_by_*.png",
    "bar_recall_by_*.png",
    "bar_f1_by_*.png",
    "cm_*_*.png",
]

# проверка critical #
missing_critical = [str(p) for p in critical if not Path(p).exists()]
if missing_critical:
    raise AssertionError(f"Отсутствуют обязательные файлы: {missing_critical}")

# сводка по паттернам #
summary = {}
for pat in patterns:
    files = list(FIG_DIR_03.glob(pat))
    summary[pat] = len(files)

# печать сводки #
print("[fairness] report @", FIG_DIR_03)
for pat, cnt in summary.items():
    print(f"    {pat:28s} -> {cnt:3d}")

# дополнительная логика: предупреждения #
warn = []

# если есть чувствительные признаки, ожидаем как минимум по одному графику var_* #
_sens_df = A.get("sensitive", (globals().get("X_test_sensitive", None), None))[0]
present_cols = [
    c for c in ["sex", "race", "age_group"] if (_sens_df is not None and c in _sens_df.columns)
]
for col in present_cols:
    for base in ["bar_precision_by_", "bar_recall_by_", "bar_f1_by_"]:
        if not list(FIG_DIR_03.glob(f"{base}{col}.png")):
            warn.append(f"Нет {base}{col}.png")

# хотя бы одна матрица ошибок #
if summary.get("cm_*_*.png", 0) == 0:
    warn.append("Нет confusion matrices (cm_*_*.png)")

# хотя бы один calibration_*.png #
if summary.get("calibration_*.png", 0) == 0:
    warn.append("Нет calibartion_*.png")

if warn:
    print("[fairness][warn]", "; ".join(warn))
else:
    print("[fairness] OK: полный набор файлов сформирован.")